# Using this Notebook to scrape Soundcloud to get a list of users that someone is following and who is following that use

Soundcloud has 176 million users. Let's try to capture that

In [7]:
import requests
import pandas as pd
import pprint
from time import sleep

### This is for getting the list of people that a user is following

#### These are all the functions that need to be ran before the notebook works:
1. Getting user information (this is used for following and follower list)

In [64]:
# user_id = '58899599'

# following_url = f'https://api-v2.soundcloud.com/users/{user_id}/followings?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'


# following_parsed = requests.get(following_url).json()
# print(following_parsed.keys())
# print(following_parsed.get('next_href'))


# following = get_user_info(following_parsed)

# follower_url = f'https://api-v2.soundcloud.com/users/{user_id}/followers?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535640551&app_locale=en'
# follower_parsed = requests.get(follower_url).json()
# followers = get_user_info(follower_parsed)


In [8]:
def get_user_info(user_id, list_subject):
    """
    this function gets all the information for the json file that is parsed from requests.
    list_subject tells me whether they want following list or follower list
    I need to also account for the fact that next_href can be NOT None so I need to rerun the fcn until that is true
    """
    
    
    url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset=0&linked_partitioning=1&app_version=1535972763&app_locale=en'
    parsed = requests.get(url).json()
    following_list = []
    base_url = ''
    
    while parsed.get('next_href') != None: 
        following_list.extend(parsed.get('collection'))
        offset = parsed.get('next_href').split('offset=')[1].split('&')[0]
        url = f'https://api-v2.soundcloud.com/users/{user_id}/{list_subject}?client_id=STLIEkQm2mwaCsHNwUkZgUIDzO5FiGTY&limit=200&offset={offset}&linked_partitioning=1&app_version=1535972763&app_locale=en'
        parsed = requests.get(url).json()

    following_list.extend(parsed.get('collection'))
    print(len(following_list))
    return following_list #this is the collection list which includes all the users in the list_subject

Here's the plan:
- everytime I get someone's follower's, i'm going to put their user_id into a list.
- before i get someone's followers, i need to find if that user_id is already in the list
- - and if not, then i will scrape. if so, then i will not scrape

In [ ]:
# here is a template of how to write things to a csv

# import csv
# RESULT = ['apple','cherry','orange','pineapple','strawberry']
# with open("output.csv",'wb') as resultFile:
#     wr = csv.writer(resultFile, dialect='excel')
#     wr.writerow(RESULT)

In [90]:
# i need to keep track of all the people that i've already scraped

following = get_user_info('38611053', 'followers')
got_followers = []
got_followers.append('38611053')
#print(type(got_followers))
got_followers_df = pd.DataFrame(got_followers).to_pickle('got_followers.pkl') # this is the file where I am storing scraped
#got_followers_df

212


In [15]:
mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count', 'permalink',\
             'reposts_count', 'track_count', 'uri']

following_df = pd.DataFrame(following)[mask]

In [16]:
mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count', 'permalink',\
             'reposts_count', 'track_count', 'uri']


following_df
# things that I want: followers_count, followings_count, id, likes_count, permalink, reposts_count, track_count, uri

,city,country_code,followers_count,followings_count,id,likes_count,permalink,reposts_count,track_count,uri
0,None,None,7,6,477797010,7,notatyourparty,None,0,https://api.soundcloud.com/users/477797010
1,"garden grove, ca",None,9,34,496307469,1,prettylit,None,1,https://api.soundcloud.com/users/496307469
2,None,None,6,10,176660107,10,brandon-bun-699663406,None,0,https://api.soundcloud.com/users/176660107
3,None,None,1,3,440219016,1,oliver-trombetti,None,0,https://api.soundcloud.com/users/440219016
4,None,None,19,8,276222017,3,user-713335820,None,18,https://api.soundcloud.com/users/276222017
5,Vancouver,CA,1439,531,202142770,203,ngynmusic,None,18,https://api.soundcloud.com/users/202142770
6,None,None,66,66,26437110,49,claudio-labra,None,0,https://api.soundcloud.com/users/26437110
7,None,None,9,328,416767671,623,whiskeythanyourtea,None,0,https://api.soundcloud.com/users/416767671
8,None,None,0,3,429935160,2,sdred-abramowitz,None,0,https://api.soundcloud.com/users/429935160
9,None,None,7,31,116322161,1,jennifer-phvm,None,0,https://api.soundcloud.com/users/116322161


In [70]:
# list(following_df.id)

In [71]:
list_of_users_seen = []
list_of_users_seen.extend(list(following_df.id))
list_of_users_seen = set(list_of_users_seen) #doing set makes sure that i dont have duplicates
len(list_of_users_seen)

212

In [72]:
# test_list = []
# test_list.extend(['a', 'b', 'c'])
# test_list.extend(['a', 'b', 'c'])

In [111]:
import numpy as np
got_followers= np.array([])

### This is for getting the list of followers of a user:

In [ ]:
# test
import pickle
followers = get_user_info('38611053','followers')
follower_df = pd.DataFrame(followers)[mask]
got_followers = np.append(got_followers,'38611053')
print(got_followers)
got_followers_df = pd.DataFrame(got_followers).to_pickle('got_followers.pkl')


follower_df

# move things into the env file

In [9]:
from pymongo import MongoClient
# !sudo apt-get install -y mongodb-org

In [5]:
!sudo service mongod start


In [7]:
# !mongo -u mongo_user -p password 127.0.0.1/cool
import os
import dotenv
%load_ext dotenv
%dotenv

In [11]:
connection_string =os.environ['MONGO_CONNECTION_STRING']

In [12]:
# i need to put this into a .env file so that i dont get blocked like billy did :(((
# user_followers is the database that i created to put my information into it

#connection_string = 'conntection_string'

client = MongoClient(connection_string)

In [19]:
#this is me creating a newdatabase

database = client.user_followers

followers = database.followers
following = database.following
errors = database.errors

The format of followers is that I will start with the source node, destination node, and the relationship is 'following'

In [10]:
# list(following.find())

In [11]:
# following.insert_one({'name': 'jenn', 'who': 'everyone'})
#followers.insert_one({'name': 'brett', 'who': 'everyone'})

In [17]:
db = client.user_followers
followers = db.user_followers
test = db.test

In [15]:
db.list_collection_names()

['following', 'user_followers', 'followers']

Recall that I have a function: 'get_user_info('user_id', 'list_subject') that returns a collections list.
1. I want to use the mask:

mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count', 'permalink','reposts_count', 'track_count', 'uri']

In [14]:
# these are the keys that i actually care about
mask = ['city', 'country_code', 'followers_count', 'followings_count', 'id', 'likes_count',\
        'permalink','reposts_count', 'track_count', 'uri']

#initiating an empty set that i will use as my hash
already_scraped = set()
queue_of_followers = set()

In [27]:
# this is my seed. I will be using this user to get a list of people to find their followers!
user_id = '38611053'
pd.to_pickle(set([user_id]), 'already_scraped.pkl')
pd.to_pickle((['38611053']), 'queue_of_followers.pkl')
# start off with having a queue and hash
queue = set(pd.read_pickle('queue_of_followers.pkl'))
hash_table = pd.read_pickle('already_scraped.pkl')


the following function mongo_user_followers does:
1. gets a user's followers
2. removes the user from the queue, and adds it to the hash
3. saves the new hash
4. updates the queue with the new followers its discovered
5. saves the queue (this is a precaution just in case my stupid instance dies and i have to figure out where i am)

In [38]:

# i need to check if the user id is in the hash before i enter this function, becaues i dont account for it
def mongo_user_followers(user_id):
    print(user_id)
    followers_list = get_user_info(user_id, 'followers')
        
    hash_table.add(user_id) #adding the userid to the set #know the difference between add and update
    pd.to_pickle(hash_table, 'already_scraped.pkl')

    followers_df = pd.DataFrame(followers_list)[mask] # to get the columns that i actually care about
    
    # now i have more followers that i need to add to the queue
    queue.extend((followers_df.id.tolist()))
    pd.to_pickle(queue, 'queue_of_followers.pkl')
    
    #pd.to_pickle(queue_of_followers, 'queue_of_followers.pkl')
    followers_df['is_following'] = user_id #this is the userid that you entered into the function
    followers_list = followers_df.to_dict(orient = 'records')
    print(f"Hello human, I've got the follower_dict for {user_id}")
    sleep(0.2);
    return followers_list

def add_to_mongo(list_of_users):
    followers.insert_many(list_of_users)


In [30]:
# queue = (pd.read_pickle('queue_of_followers.pkl'))
# print(queue)
# print(type(queue))
# hash_table = pd.read_pickle('already_scraped.pkl')
# followers_dict = mongo_user_followers('38611053')


# #this adds the list of followers to the mongodb collection
# add_to_mongo(followers_dict)

['38611053']
<class 'list'>
38611053
212
Hello human, I've got the follower_dict for 38611053


In [44]:
len(queue)
print(hash_table)

{'38611053'}


So now that I've added it to my mongo, I need to read the queue to see who I need to get next.
I think for the queue, I need to use a list or np.array or something

In [42]:
# i need to initialize the queue and hash variables so that i have something to start with and add to

queue = (pd.read_pickle('queue_of_followers.pkl')) # this is a list
hash_table = pd.read_pickle('already_scraped.pkl') #this is a set


while (len(queue) !=0) and (len(hash_table) <=60000):
    next_user = queue.pop(0)
    if next_user in hash_table:
        continue
    #this function updates queue, and hast_table
    followers_dict = get_user_info(next_user) 
    add_to_mongo(followers_dict)

# this gives me the next person
#queue.remove(queue[0]) #this removes the person from the queue jk the function does it for me

#next i would need to call the function in this userid and continue to do the thing

False

In [33]:
queue

[477797010,
 496307469,
 176660107,
 440219016,
 276222017,
 202142770,
 26437110,
 416767671,
 429935160,
 116322161,
 142116091,
 181520129,
 44956027,
 78551230,
 38504922,
 58899599,
 144284855,
 59931602,
 101100987,
 25834779,
 71702114,
 67603860,
 229265510,
 166834082,
 144901714,
 18088787,
 281433942,
 125244985,
 322402288,
 335629542,
 165812516,
 331960939,
 163685788,
 111298931,
 318606620,
 282001115,
 27687218,
 169412367,
 46609448,
 280117747,
 300098585,
 176619955,
 217114241,
 240933759,
 56349681,
 262598350,
 174953431,
 100178292,
 168471375,
 18112950,
 12705069,
 47016505,
 224634872,
 261918552,
 22406196,
 1857457,
 290266964,
 144131482,
 50186462,
 54502132,
 235068602,
 207879,
 61390123,
 13759019,
 12167698,
 249579736,
 76694815,
 154024450,
 8999742,
 113562763,
 22687695,
 131395308,
 166870212,
 87163326,
 227595690,
 45922515,
 67680777,
 150743920,
 104820979,
 16058350,
 39427745,
 14053163,
 172611694,
 41348361,
 269374072,
 34932799,
 364494

In [40]:
# delete = list(followers.find())
# for user in delete:
#     followers.delete_one(user)